# 모델 학습

## Background

- 텐서플로우
  - Google 개발
  - 사용이 복잡했으나 2.0이후 개선
  - colab연동 쉬움
  - Tensorboard를 통한 강력한 시각화

- 파이 토치
  - Facebook 개발
  - 일부 모델의 구현이나 데이터 처리에 더 적합
  - 모델의 자세한 구조나 연산을 연구하는 용도로 적합

- Tensorflow 2.7
  - keras 기능들이 안정적으로 포함됨
  - 다양한 구조의 모델을 간편하게 구현 가능하고 사용자 정의 구조를 정의가 개선됨
  - GPU를 사용할 수 있는 환경에서 자동으로 데이터를 GPU에 업로드 하는 과정을 수행
  - LSTM을 포함한 구조의 연산을 최적화함

- 과적합이란 무엇인가
  - 모델이 데이터의 일반적인 특징이 아닌 학습 데이터에 특화되어 학습
  - 과적합의 원인
    1. 데이터의 수나 다양성이 부족하다.
    2. 데이터에 비해 모델이 너무 크다.
    3. 너무 많은 epoch을 학습했다.
  - 해결방안
    1. 데이터를 늘린다.
    2. 과적합이 일어나기 전에 학습을 정지한다(Early stop)
      - 검증 성능이 가장 높은 시접에서 학습을 중단하고 모델을 사용
    3. 데이터 증강(Data augmentation)
      - 같은 데이터도 반전시키거나 변형해서 입력
      - 적은 수의 데이터를 이용하여 좀더 일반적인 특징을 학습하도록 유도

### fit 함수

```python
fit(
  x=None, y=None, batch_size=None, epochs=1, verbose='auto',
  callbacks=None, validation_split=0.0, validation_data=None,
  suffle=True, class_weight=None, initial_epoch=0, 
  steps_per_spoch=None, validation_steps=None, 
  validation_batch_size=None, validation_freq=1,
  max_queue_size=10, workers=1, use_muliprocessing=False
)
```
- 모델을 학습하는 핵심함수
  - 다양한 옵션을 제공하는 인수를 포함하고 있음
  - 모델 학습을 위해서는 각 인수의 의미를 파악할 필요가 있음
  - 이전 버전에서 사용하던 git_generator 함수도 포함되었음

- 학습 데이터
  - x : 학습 세트의 입력 데이터
  - y : Label, Ground truth, 학습 세트의 입력 데이터를 넣었을 때 정답 값

- 학습 진행도를 출력하는 방법
  - verbose: 학습 과정을 출력하는 방법
    - 0: 출력하지 않음
    - 1: 진행바를 표시하여 진행상황을 표시
    - 2: 진행바 생략, 수치정보만 표시

- Epoch과 batch
  - epochs: 학습 데이터를 몇 번 학습할 것인지
  - batch_size : 한번에 입력할 데이터 수
    - 모델의 input_shape도 이 값을 고려하여 설정
  - steps_per_epoch: 한 epoch은 몇 번의 입력을 수행하는지
    - None으로 전달하면 batch_size에 따라 자동으로 설정됨
    - batch_size * steps_per_epoch = 학습 데이터 수

- 학습 데이터 순서를 섞기
  - shuffle : 학습데이터를 섞을지 여부
    - 정해진 학습 데이터의 입력 순서에 맞춰 학습되지 않도록 순서를 변경
    - Generator를 사용할 경우 옵션에 관련 내용이 이미 존재하므로 무시됨

- 검증 데이터
  - validation_data : 검증 데이터 셋
    - (x_val, y_val)의 형태로 전달하거나 Generator로 전달
    - 이 데이터는 학습에 반영하지 않고 현재 epoch의 성능을 평가하기 위해 사용
  - validation_freq : 검증하는 주기
    - 1(기본값) : 매 epoch마다 검증
    - 자주 수행할수록 걸리는 시간은 증가하지만, 세밀하게 성능을 평가할 수 있음


- 검증 과정의 epoch, batch
  - validation_batch_size: 한번 입력에 입력할 데이터의 수
  - validation_steps : 한번 검증할 때 데이터 입력의 횟수
  - validation_batch_size * validation_steps : 검증 데이터의 수

- 데이터 불균형을 위한 가중치
  - class_weight: 클래스별 반영 정도
    - 데이터 불균형 해결하는 방법 중 하나
    - 적은 수의 클래스에는 더 많은 관심을 기울이는 개념
    - 클래스별 데이터의 수가 차이날 경우, 적은 수의 클래스는 더 많이 반영하여 학습
    - {클래스index: 반영비율}의 딕셔너리 문제를 형태로 전달
      - 예) 개(0) 100 마리, 고양이(1) 50마리
      - {0:0.5, 1:1.0}

- 데이터 병렬 처리
  - workers, use_multiprocessing : 병렬 처리를 위한 인수들
  - GPU를 사용할 경우 데이터를 읽고 변환하는 과정이 학습하는 시간보다 길게 소모함GPU는 그동안 기다리는 idle time이 발생하면서 효율이 저하
  - 여러 Workers가 동시에 데이터를 불러오고 학습을 진행
  - Generator를 사용하는 경우에만 사용

- 콜백함수의 리스트
  - Callback function
    - 다른 코드의 인수로 함수를 넘겨주면, 그 코드가 필요에 따라 실행하는 함수
    - callbacks 옵션으로 함수들의 리스트를 전달하면 학습 과정 중에 Tensorflow 가 실행
    - 다양한 함수로 학습과정을 세밀하게 조정하거나, 중간 경과를 살펴볼 수 있음

## 콜백 함수

- 콜백함수 정의하기
- keras.callbacks.Callback을 상속받아서 클래스 정의
- 호출될 타이밍에 따라 해당 함수를 재정의
  - on_train_end(self, logs=None): 학습이 종료될 때 호출
  - on_epoch_end(self, epoch, logs=None): 한 epoch이 끝날 때 호출
  - op_predict_end(self, logs=None):예측이 끝날 때 호출

- 정의한 클래스의 인스턴스를 리스트에 포함시켜 callbacks에 전달

```python
class MyCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    keys = list(logs.keys())
    print(f"End epoch {epoch} of training; got log keys: {keys}")

model.fit( ... , callbacks=[MyCallback()])
```

- 내장 콜백함수
  - Tensorflow는 다양한 콜백함수를 제공
  - tf.keras.callbacks 에 포함된 주요 콜백함수
    - EarlyStopping: 학습이 진전이 없을 경우 조기에 학습을 종류
    - ModelCheckpoint: 모델을 주기적으로 자동 저장
    - TensorBoard: 학습 과정이나 모델의 정보를 시각화 할 수 있도록 로그를 기록

### EarlyStopping

```python
tf.keras.callback.EarlyStopping(
  monitor='val_loss', min_delta=0, patience=0, verbose=0,
  mode='auto', baseline=None, restore_best_weights=False
)
```

- EarlyStopping 의 개념
  - 일정 시점부터 학습 성능이 증가하지만, 검증 성능이 감소하거나 유지되는 과적합
  - 과적합이 시작되면 더 학습을 징행할수록 성능이 감소함
  - 과적합이 발생하는 Epoch를 감지하고 자동으로 학습을 종료하는 방법

  - 모델 학습의 목표는 Loss값을 최소화 하는 과정
  - Loss값을 감시하고 있다가 더 이상 감소하지 않는다명 학습이 종료됨
  - 과적합을 방지하고 무의미한 학습과정을 생략하여 연산자원을 절약

- monitor : 감지할 수치의 이름
- mode : 원하는 방향
  - monitor 값이 어떻게 되어야 하는 값인지 결정
  - 'min': monitor 값이 최서가 되어야 성능이 좋아지는 모델일 때
  - 'max': monitor 값이 최대가 되어야 성능이 좋아지는 모델일 때

- patience : 종료 시점, 바로 종료할지, 몇번 더 학습 해볼것인지

### ModelCheckpoint

```python
tf.keras.callback.ModelCheckpoint(
  filepath, monitor='val_loss', verbose=0, save_best_only=False,
  save_weights_only=False, mode='auto', save_freq='epoch',
  options=None, **kwargs
)
```

- ModelCheckpoint
  - 일정 주기마다 모델이나 모델의 가중치를 자동으로 저장
  - 과적합이 발생했을 때 처음부터 수행하지 않고 중간부터 다시 학습이 가능
  - 선을 그래프를 보면서 가장 좋은 성능의 가중치를 서비스에 사용
  - 가장 성능이 좋은 버전만 저장하는 기능이 포함

- monitor, mode
  - 감지할 지표와 모드: EarlyStopping과 동일

- save_best_only : 가장 좋은 버전만 저장
  - True: 성능이 가장 좋은 버전만 남기고, 다른 버전 모두 삭제
  - False: 모든 버전을 저장
- save_weights_only: 모델의 가중치의 값만 저장
  - True :학습 진행상황, 모델의 구조에 대한 데이터를 빼고 저장하여 공간이 절약되지만 불러오기 위해서는 모델의 구조를 저장하고 있어야함(False 추천)

- ModelCheckpoint의 주의할 점
  - filepath: 체크 포인트를 저장할 디렉토리
    - 고정된 문자열을 전달하면 하나의 디렉토리에 계속 저장
    - Epoch에 따라 이름을 다르게 하도록 저장
    - 포맷 문자열을 사용
      - `filepath = 'checkpoints/cp-{epoch:04d}.ckpt`

### TensorBoard

- 모델의 정보를 시각화하는 TensorBoard를 사용할 수 있도록 로그를 기록
  - 실시간 학습상황
  - 모델의 구조

- 장점
  - 원격에서 실시간으로 모델의 학습정보를 포함한 정보를 확인
  - 간단하게 Callback을 추구하고 시각화된 웹페이지를 제공
  - 포트와 IP를 설정하면 서버가 아닌 원격에서 이 결과물을 활용 가능
- 사용과정
  - Callback에 TensorBoard 콜백함수를 추가하여 로그 디렉토리에 기록을 저장
  - TensorBoard 모듈을 실행하여 로그 디렉토리의 로그를 웹페이지로 호스트
  - 웹페이지에 출력된 정보들을 보고 학습 과정을 모니터링

- TensorBoard 콜백함수의 인자
  - log_dir : 로그를 저장할 경로
  - update_freq : 저장하는 주기
    - 'epoch' : 한 epoch마다 기록을 저장
    - 'batch' : batch마다 기록을 저장(학습 속도가 느려짐)

```python
tf.keras.callbacks.TensorBoard(
  log_dir='logs', histogram_freq=0, write_graph=True,
  write_images=False, write_steps_per_second=False,
  update_freq='epoch', profile_batch=0, embeddings_freq=0,
  embeddings_metadata=None, **kwargs
)
```

- Fit 함수에 콜백함수 추가하기
```python
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
model.fit(x_train, y_train, epochs=2, callbacks=[tensorboard_callback])
```

- 명령줄에서 tensorboard를 실행
  - '--logdir'에 저장한 디렉토리 경로를 전달
```bash
tensorboard --logdir=path_toyour_logs
```



# 모델 서비스 하기

## 모델 저장하고 불러오기

- 모델의 구성요소
  - 모델의 구조
    - 레이어의 종류와 형태
    - 입력 값의 형태
  - 가중치 값
    - 각 레이어의 행렬에 저장된 실제 float32 실수 값들
    - 모델의 학습 = Loss값이 낮아직도록 가중치의 값을 수정하는 과정의 연속
    - 같은 모델도 가중치 값에 따라 성능이 달라짐
  - Compile 정보
    - Optimizer의 종류, Learning Rate(lr), 사용한 Loss Function 정보

- 모델의 저장 형식
  - H5 Format
    - 과거 Keras에서 사용하던 저장 방식
    - 모델의 구조와 가중치를 포함한 정보들을 저장
    - 사용자 정의 레이어와 손실함수는 저장하지 않음

- SavedModel
  - 최근에 사용하는 Tensorflow 표준 저장 형식
  - 모델의 구조와 가중치를 포함한 정보들을 저장
  - 사용자 정의 레이어와 손실함수까지 모두 저장

- SavedModels 활용 방법
  - 모델 저장

```python
model.fit(x, y)
#'my_model'이라는 이름의 savedModel 폴더를 생성
model.save('my_model')
```

  - 저장된 모델 사용

```python
#'my_model'이라는 이름의 savedModel 폴더를 생성
loaded_model = keras.models.load_model('my_model')
```

- 이어서 학습하기
  - SavedModel 형식은 모델의 모든 정보를 저장하고 불러오는 방식
  - 불러온 모델을 그대로 학습을 마저 진행
    - initial_epoch과 epoch을 조장하여 이어서 학습을 진행

```python
model.comile(...)
model.fit(x, y)
# 모델을 compile하고 학습까지 진행하고 저장
model.save('my_model')

loaded_model = keras.models.load_model('my_model')
# 저장한 시점의 compile 정보와 학습 상태까지 불러옴
# 다시 compile 하지 않아도 바로 이어서 학습 가능
# 21 epoch부터 40 epoch 까지 계속 학습
loaded_model.fit(x, y, initial_epoch = 20, epochs = 40)
```

- Checkpoint 불러오기
  - Checkpoint 콜백함수의 인수 중 save_weights_only를 False로 설정
  - 원하는 epoch의 체크포인트 경로를 전달
  - initial_epoch과 epoch을 조절하여 이어서 학습을 진행

```python
# 모델을 학습하면서 Checkpoint 를 저장
# 20epoch 이후 저장된 체크포인트를 불러옴
loaded_model = keras.models.load_model("checkpoints/cp-0020.ckpt")
# 21 epoch부터 20 epoch까지 계속 학습
loaded_model.fit(x, y, initial_epoch = 20, epochs = 40)
```

## 모델 서비스 방법

- 자바스크립트를 이용한 모델 서비스
  - Tensorflow.js
    - 자바스크립트 Tensorflow 라이브러리
    - 브라우저 또는 Node.js 에서 학습된 모델을 사용가능
  - Tensorflow.js에서 사용하는 과정
    - 모델의 학습을 Tensorflow에서 진행
    - Python 에서 학습된 모델을 Tensorflow.js에 맞는 형식으로 변환
    - Tensorflow.js 에서 모델의 동작을 js로 작성하여 서비스

- Tensorflow.js 형식으로 변환하는 방법
  - tensorflowjs 를 사용하여 변환하여 저장
  - target_dir에 json파일을 포함한 변환된 모델이 저장됨

```python
import tensorflowjs as tfjs
def train(...):
  model = keras.models.Sequential()
  ...
  model.compile(...)
  model.fit(...)

  tfjs.converters.save_keras_model(model, target_dir)
```

- Tensorflowjs로 변환한 모델 사용방법
  - model.json 파일의 URL을 제공하여 TensorFlow.js에 모델을 로드

```js
import * as tf from '@tensorflow/tfjs';
const model = swait
tf.loadLayersModel("https://foo.bar/tfjs_artifacts/model.json")
```

- 모델을 학습, 추론하기

```js
const example = tf.fromPixels(webcamElement);  // 웹캠 Element를 사용한다고 가정
const prediction = model.predict(example);     // 예측에 사용
```

- Flask에서 서비스하기
  - 학습한 모델을 불러오기 : SavedModel 형식의 모델을 로드

```python
if __name__ == '__main__':
  model = tf.keras.models.load_model("my_model")
  app.run(host='localhost', port=8000)
```

  - 모델의 사용은 그대로 predict를 사용

```python
res = model.predict([inputdata])
# res에서 정보를 추출하여 서비스에 활용
```

## 서버 안정화 처리

- 딥러닝 모델 서비스
  - 모델의 추론과정은 일반적인 연산에 비해 처리 시간이 길고 자원도 많이 소모
  - GPU자원을 사용하는 경우 GPU를 병렬적으로 사용하기 위한 처리가 필요
  - 사용자의 요청에 따라 계속 연산을 처리하면 서비스가 종료될 수 있음

- 서비스 안정화
  - 사용자의 요청을 거절하더라도 서비스가 종료되지 않도록 자원관리가 필요
  - 서버의 연산 성능, 처리중인 작업의 수를 고려한 설계가 요구됨

- 실행 가능한 작업 제한
  - 서버에서 동시에 진행가능한 작업의 수를 제한하여 안정화
  - 구현 방법의 예시
    - 동시에 처리 가능한 최대 작업의 수를 상수로 정의(max_works)
    - 전역변수를 이용하여 진행중인 작업의 수를 저장(num_works)
    - 사용자의 요청이 왔을 때, 진행중인 작업의 수 비교(max_works > num_works)
      - 작업의 수가 최대일 때 : 작업을 수락하고 num_works를 1 증가 시킴
      - 작업의 수가 최대가 아닐 때 : 작업을 수락하고 num_works를 1 증가시킴
    - 작업이 완료되면 작업 수를 1 감소(num_works -= 1)

  - 장점
    - 비교적 간단하게 구현이 가능
    - 가벼운 모델만 사용할 때, 서비스가 다운되는 현상은 방지할 수 있음
  - 단점
    - 사용자가 작업을 예약하는 등의 처리는 어려움
    - 작업이 매우 오래 걸리는 모델의 경우 응답까지 시간이 오래걸림
    - 대규모 서비스에 작용하기는 부적절함

- 작업 큐를 이용한 비동기 처리
  - 사용자와 상호작용을 관리하는 프로세스와 작업을 처리하는 Worker프로세스를 분리
  - 각 프로세스는 큐를 이용하여 상호작용함
  - 유사한 방식이 활용되는 분야
    - 안드로이드 GUI의 이벤트 시스템
    - Windows의 메시지 큐와 윈도우 프로시저
    - 그래픽 처리 분야의 렌더링 큐
    - 프린터의 스풀링작업
    - Tensorflow의 데이터를 불러오기 위한 workers 옵션











